In [ ]:
from web3 import Web3
import os
from dotenv import load_dotenv
import pandas as pd
import prophet
import numpy as np
import datetime as dt
from datetime import timedelta
from prophet import Prophet
from eth_account import Account
from web3.middleware import geth_poa_middleware
from eth_abi import decode
from eth_utils import decode_hex, to_text

import requests
import random
import json

from dune_client.client import DuneClient

import plotly.graph_objs as go

In [ ]:
load_dotenv()

ETHERSCAN_KEY = os.getenv("ETHERSCAN_KEY")
COINGECKO_API_KEY = os.getenv("COINGECKO_API_KEY")
DUNE_API_KEY = os.getenv('DUNE_API_KEY')
FLIPSIDE_API_KEY=os.getenv('FLIPSIDE_API_KEY')
dune = DuneClient(DUNE_API_KEY)

GAS_RESERVE = os.getenv('GAS_RESERVE')
ACCOUNT_ADDRESS = os.getenv("PROTOCOL_CONTROLLER")
PRIVATE_KEY = os.getenv("PROTOCOL_CONTROLLER_KEY")
YIELD_FARM_ADDRESS = os.getenv("YIELD_FARM_ADDRESS")
STAKING_CONTRACT = os.getenv("STAKING_CONTRACT")
SEPOLIA_GATEWAY = os.getenv("SEPOLIA_GATEWAY")

In [ ]:
os.chdir('..')

api = False

In [ ]:
abi_path = r'gas_accountant_contracts\contracts\artifacts'
abi_paths = []  # Assuming GAS_ACCOUNTANT_ABI_PATH is predefined

for file in os.listdir(abi_path):
    if file.endswith('.json') and "metadata" not in file:  # Exclude metadata files
        abi_paths.append(os.path.join(abi_path, file))  # Add full path

print(abi_paths)  # Debug: Check the final list

abis = {}

for path in abi_paths:
    filename = os.path.basename(path)  # Extract filename (e.g., "YieldVault.json")
    name = os.path.splitext(filename)[0]  # Remove .json extension (e.g., "YieldVault")

    with open(path, "r") as file:
        abis[name] = json.load(file)  # Use name as key

print(abis)  # Debug output

In [ ]:
w3 = Web3(Web3.HTTPProvider(SEPOLIA_GATEWAY))

ACCOUNT = Account.from_key(PRIVATE_KEY)

w3.eth.default_account = ACCOUNT.address
w3.middleware_onion.inject(geth_poa_middleware, layer=0)